# Тема 2.4.4 Использование обученных моделей RusVectōrēs для определения степени схожести текстов.  




В предыдущей теме мы рассмотрели сервис [RusVectōrēs](https://rusvectores.org/ru/about/), который вычисляет семантические отношения между словами русского языка и позволяет скачать предобученные дистрибутивно-семантические модели (word embeddings).  

В данной теме применим готовые модели для определения степени схожести текстов. Это позволит, например, искать ответ на вопрос.  
Для примера рассмотрим датасет вопросов форума Поволжского государственного технологического университета. Датасет собран и размечен вручную экспертами.  

In [248]:
# импортируем все необходимые зависимости.
import warnings
import logging
from typing import Dict
import zipfile
import wget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gensim
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import metrics
from utils import reset_random_seeds


# Общие настройки.
RANDOM_SEED = 42
reset_random_seeds(RANDOM_SEED)
plt.style.use('ggplot')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
warnings.filterwarnings('ignore')

В папке `'./data/volgatech_faq'` представлен датасет:  
Описание файлов данных:  
    `questions.csv` - вопросы пользователей;  
    `answers.csv` - ответы на вопросы;  
    `pos_relations.csv` - правильные ответы на вопросы (пары вопрос==ответ);  
    `neg_relations.csv` - неправильные (неподходящие) ответы.  

In [2]:
!ls ./data/volgatech_faq

answers.csv       neg_relations.csv questions.csv
data.zip          pos_relations.csv


In [228]:
df_questions = pd.read_csv('./data/volgatech_faq/questions.csv', sep=';', names=['id', 'text'])
df_questions.head()

,id,text
0,3647,размерность векторного пространства
1,3644,расстояние
2,3643,привет
3,3631,что такое подпространство
4,3630,матрица перехода к новому базису


In [229]:
# Посчитаем количество уникальных текстов
df_questions['text'].nunique()

92

In [230]:
df_answers = pd.read_csv('./data/volgatech_faq/answers.csv', sep=';', names=['id', 'text'])
df_answers.head()

,id,text
0,339,Размерностью векторного пространства (ВП) назы...
1,358,С Евклидовым расстоянием мы с вами хорошо знак...
2,383,Данный форум не предназначен для решения орган...
3,384,Данный форум не предназначен для решения орган...
4,374,Опр. Подпространством линейного пространства V...


In [233]:
# Посчитаем количество уникальных текстов
df_answers['text'].nunique()

32

In [231]:
# Для удобства работы далее нам нужно отсортировать значения таблиц вопросов и ответов по колонке 'id'

df_answers = df_answers.sort_values(by=['id'], axis=0, ignore_index=True)
df_questions = df_questions.sort_values(by=['id'], axis=0, ignore_index=True)

In [232]:
df_answers.head()

,id,text
0,327,"Векторные пространства, в которых задано скаля..."
1,329,"А сейчас отметим следующее, что, задав скалярн..."
2,331,"Это поле рациональных чисел, с которыми мы фак..."
3,334,Давайте введем понятие абстрактного векторного...
4,336,Кроме геометрических векторов - направленных о...


In [284]:
df_positive = pd.read_csv('./data/volgatech_faq/pos_relations.csv', sep=';', names=['question', 'answer'])
df_positive.head()

,question,answer
0,3647,339
1,3644,358
2,3643,"383,384"
3,3631,374
4,3630,405


В данных 1 вопросу может соотвествовать несколько ответов.  
Изменим данные - так чтобы 1 вопросу соответствовал 1 ответ для упрощения нашего учебного примера, оставим только первый ответ.  

In [285]:
df_positive['answer_1'] = df_positive['answer'].apply(lambda t: [int(a.strip()) for a in t.split(',')][0])

In [286]:
df_positive.head()

,question,answer,answer_1
0,3647,339,339
1,3644,358,358
2,3643,"383,384",383
3,3631,374,374
4,3630,405,405


Загрузим модель RusVectores.  
В данных могут встречаться редкие слова, опечатки и прочее. Целесообразно использовать модель на основе fasttext.  

In [22]:
MODEL_FT_URL = 'http://vectors.nlpl.eu/repository/20/181.zip'

model_arch_file = MODEL_FT_URL.split('/')[-1]
model_path = model_arch_file.split('.')[0]

In [23]:
# Скачивание модели.
# размер файла - 2,6 Gb.
# Не выполняйте этот код повторно, если модель уже скачана и распакована.

# _ = wget.download(MODEL_FT_URL)
# print(f'extract {model_arch_file} to path: {model_path}')
# with zipfile.ZipFile(model_file, 'r') as archive:
#     zipfile.ZipFile.extractall(archive, model_path)

In [24]:
vectorizer_ft = gensim.models.KeyedVectors.load(f'{model_path}/model.model')

2022-01-24 09:41:13,448 : INFO : loading Word2VecKeyedVectors object from 181/model.model
2022-01-24 09:41:14,188 : INFO : loading vectors_vocab from 181/model.model.vectors_vocab.npy with mmap=None
2022-01-24 09:41:14,424 : INFO : loading vectors_ngrams from 181/model.model.vectors_ngrams.npy with mmap=None
2022-01-24 09:41:17,641 : INFO : loading vectors from 181/model.model.vectors.npy with mmap=None
2022-01-24 09:41:17,964 : INFO : setting ignored attribute vectors_vocab_norm to None
2022-01-24 09:41:17,965 : INFO : setting ignored attribute buckets_word to None
2022-01-24 09:41:17,967 : INFO : setting ignored attribute vectors_norm to None
2022-01-24 09:41:17,971 : INFO : setting ignored attribute vectors_ngrams_norm to None
2022-01-24 09:41:17,972 : INFO : loaded 181/model.model


In [345]:
def get_text_vector(text: str, model: gensim.models.KeyedVectors = vectorizer_ft) -> np.array:
    """
    Получить усредненный вектор слов текста.

    Args:
        text (str): текст.
        model (gensim.models.KeyedVectors): модель.

    Returns:
        np.array: результат.
    """

    tokens = text.split()
    v = np.mean([model.get_vector(t) for t in tokens], axis=0)

    return v

t = df_questions['text'].values[0]
v = get_text_vector(t, vectorizer_ft)
print('text:', t)
print('размер вектора:', v.size)
print('vector:', list(v[:5]) + ['...'] + list(v[-5:]))

text: что такое евклидово пространство
размер вектора: 300
vector: [-0.039669473, -0.06307665, 0.18722992, 0.15415101, -0.117433324, '...', -0.21156421, 0.21496902, -0.06517227, 0.3788792, 0.11184903]


Попробуем самый простой подход - посчитаем косинусное расстояние между парами вопрос/ответ.  

Чтобы не считать вектора текстов каждый раз - сохраним их в датасете.

In [282]:
df_answers['text_v'] = df_answers['text'].apply(get_text_vector)
df_questions['text_v'] = df_questions['text'].apply(get_text_vector)

Для того чтобы оценить качество алгоритма, нам нужна метрика качества.  
Применим `top_k_accuracy_score` из пакета `scikit-learn`.  
Эта метрика получает 4 основных параметра:  
`y_true` - список индексов правильных ответов.  
`y_score` - результат работы модели, вероятности ответов.  
`k` - количество ответов, среди которых должен быть 1 точный.  
`labels` - все возможные ответы.  

In [367]:
# Сделаем простой пример для понимания работы метрики.
# Допустим у нас есть всего 3 варианта ответа - 0, 1, 2
y_labels = [0, 1, 2]
# И 3 вопроса: вопрос 0 с ответом 0 и вопрос 1 с ответом 1, 2 == 2.
# Соответственно правильные ответы на вопросы [0, 1, 2] должны выглядеть так:
y_true = [0, 1, 2]
# Наша модель выдает распределение вероятностей по ответам.
y_score = [
    [0.5, 0.3, 0.1], # Наибольшая вероятность ответа 0 - правильно
    [0.3, 0.5, 0.1], # Наибольшая вероятность ответа 1 - правильно
    [0.5, 0.1, 0.3], # Наибольшая вероятность ответа 0 - НЕПРАВИЛЬНО
]

# Считаем точность попадание ответов в разные топ-К:
for k in [3, 2, 1]:
    acc = metrics.top_k_accuracy_score(y_true=y_true, y_score=y_score, k=k, labels=y_labels)
    print(f'k={k}: {acc}')


k=3: 1.0
k=2: 1.0
k=1: 0.6666666666666666


Посмотрим, что получится на всём датасете.

In [295]:
y_labels = df_answers['id'].values

y_true = [df_positive[df_positive['question'] == q_id]['answer_1'].values[0]
          for q_id in df_questions['id'].values]

y_score = cosine_similarity(list(df_questions['text_v']), list(df_answers['text_v']))

metrics.top_k_accuracy_score(y_true=y_true, y_score=y_score, k=5, labels=y_labels)



0.7967914438502673

И напишем функцию для получения топ-к ответов:


In [331]:
def get_top_k_answers(q: str, k: int = 5) -> Dict[str, float]:
    """
    get_top_k_answers [summary]

    [extended_summary]

    Args:
        q (str): Текст вопроса.
        k (int, optional): Количество ответов. Defaults to 5.

    Returns:
        Dict[str, float]: Результат в формате {текст_ответа: мера_близости}
    """

    q_vect = get_text_vector(q)
    y_score = cosine_similarity([q_vect], list(df_answers['text_v']))[0]
    result = {df_answers['text'].values[i]: y_score[i]
             for i in np.argsort(y_score)[::-1][:k]}
    return result

In [373]:
questions = [
    df_questions['text'].values[0],
    'в чём смысл жизни?',
]
for q in questions:
    print('-'*80)
    print('Вопрос:', q)
    for answer, score in get_top_k_answers(q=q).items():
        print(answer[:73]+'...', score)

--------------------------------------------------------------------------------
Вопрос: что такое евклидово пространство
Мы будем говорить, что линейное пространство конечномерно, если либо оно ... 0.7875148
Векторные пространства, в которых задано скалярное произведение называютс... 0.7598348
При решении многих практических задач нам нужно оценивать расстояние. Дав... 0.7380918
После того, как мы ввели понятие базиса мы можем ввести понятие координат... 0.73322964
Что опять же интересно – это то, что задав метрику – функцию расстояния, ... 0.73199576
--------------------------------------------------------------------------------
Вопрос: в чём смысл жизни?
После того, как мы ввели понятие базиса мы можем ввести понятие координат... 0.63086075
Мы ввели понятие абстрактного ВП. И уже начали понимать, что под ВП можно... 0.609517
Базис линейного пространства это аналог системы координат в привычной жиз... 0.6087173
Данный форум не предназначен для решения организационных вопросов. Вы мо

# Выводы 

1. Вектора, предобученные на больших корпусах текстов могут давать неплохой результат "из коробки".  
2. Подбор оптимальной модели векторизации текста может привести к значительно лучшим результатам.  
3. Вместо простого косинусного расстояния может использоваться выученная метрика близости см. metric learning.  